In [11]:
# 输入数据的参数
_INPUT4='{"name":"input4","type":0,"uri":"tmp_f54eba29f7f046f7b00bd3903ce6af5f"}'
_INPUT1='{"name":"input1","type":0,"uri":"tmp_c31422c844564b7892034c72866b69df"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_34026acc7f8d4e448b7cfa23dbd98efb"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_46b99f34deef443b90b440eb94b9c6d3"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_cdb23c6c055743189d49fa78f01f13c2"},{"name":"output2","type":0,"uri":"tmp_b3828ecffc5d4ac0821f7ef55bb160f3"},{"name":"output3","type":0,"uri":"tmp_d60899a9bb754fa4a012a7e190bb820b"},{"name":"output4","type":0,"uri":"tmp_6e9eb9d1ead54c9d9f8bbc6aa653c781"}]'

# 自定义参数


In [12]:
import pandas as pd
import wfio
import random
# from ustciscrLab_A import get_score
from ustciscrBDL_B import get_score
import math
import numpy as np
from scipy.stats import entropy

In [13]:
# 读取训练集数据1
def get_train1_data():
    train_data1 = wfio.read_dataframe(_INPUT1)
    return train_data1
# 读取训练集数据2
def get_train2_data():
    train_data2 = wfio.read_dataframe(_INPUT2)
    return train_data2
# 读取测试数据集1
def get_test1_data():
    test_data1 = wfio.read_dataframe(_INPUT3)
    return test_data1
# 读取测试数据集2
def get_test2_data():
    test_data2 = wfio.read_dataframe(_INPUT4)
    return test_data2


In [14]:
#随机产生验证集的答案，用于测试
def create_verify_train_data():
    train1 = get_train1_data()
    train2 = get_train2_data()
    train1.columns = train2.columns.tolist()
#     # 合并两个训练集
#     train = pd.concat([train1,train2],axis=0,join='inner',ignore_index=True)
    test2 = get_test2_data()
    # 总的数据集
    df = train1.append(test2).reset_index(drop=True)
    # 利用正则表达式将空字符串替换为数字0
    df.replace(to_replace=r'^\s*$',value=0,regex=True,inplace=True)
    # 把有汉字和没有汉字的列分开
    columns_list = df.columns.tolist()
    numerical = []  # 数值
    category = []   # 汉字

    # 取出前96个
    for i in range(96):
        numerical.append(columns_list[i])
    # 增加从业人数和注册资本
    numerical.extend(['cyrs','zczb'])
    for j in numerical:
         df[j] = df[j].astype(float)
    # 汉字列表
    for k in numerical:
        category = columns_list
        category.remove(k)
    jxje_list = []  #进项金额对应列名
    jxsl_list = []  #进项税率对应列名
    kpse_list = []  #开票税额对应列名
    rkse_list = []  #入库税额对应列名
    kpsl_list = []  #开票税率对应列名
    kpje_list = []  #开票金额对应列名
    kps_list  = []  #开票数对应列名
    for i in df.columns.tolist():
        if 'jxje' in i:
            jxje_list.append(i)
        elif 'jxsl' in i:
            jxsl_list.append(i)
        elif 'kpse' in i:
            kpse_list.append(i)
        elif 'rkse' in i:
            rkse_list.append(i)
        elif 'kpsl' in i:
            kpsl_list.append(i)
        elif 'kpje' in i:
            kpje_list.append(i)
        elif 'kps' in i:
            kps_list.append(i)    
        else:
            pass

    # **************************************************规则一 二 ****************************************************   
    zbq = []     #存放公司增版比率
    zlq = []     #存放公司增量风险指数
    dekjq = []    #顶额开具的季度数量

    for index, row in df.iterrows():
        dekj_cach = [] #顶额开具风险缓存列表
        zl_cach  = []    #开票增量缓存列表
        bmpe_cach = []   #存放每个季度的版面票额缓存列表
        for i in range(12):
            kpje = float(row[kpje_list[i]])    #开票金额
            kps  = float(row[kps_list[i]])     #开票数
            if kps != 0.0:
                bmpe = kpje/kps        #版面票额
                bmpe_cach.append(bmpe)
                zl_cach.append(kpje)
                if int(str(bmpe)[0]) == 9:  #版面票额最高位为9
                    dekj_cach.append(i)     #将版面票额超过版额的90%的季度存起来
        dekjq.append(len(dekj_cach))                
        #增版判断
        if bmpe_cach:
            zbq.append(max(bmpe_cach)/min(bmpe_cach))
        else:
            zbq.append(0)
        #增量判断    
        if zl_cach: 
            zlq.append(max(zl_cach)/min(zl_cach))
        else:
            zlq.append(0)
    # **************************************************规则三 ****************************************************   
    # f1_list = []      #存放公司购销风险指数
    gxfxq = []        #购销风险季度数量
    gxmonthq = []     #购销季度数量
    for index, row in df.iterrows():
        fx_list = []        #购销风险月份
        month_list = []       #存在购销月份
        for i in range(12):
            if float(row[kpje_list[i]]):   #判断存在购销的月份
                month_list.append(1)
        for i in range(12):
            if row[jxsl_list[i]] == 0.03:  #进项税率为0.03时，入库税额应该等于开票税额
                if row[kpse_list[i]] != row[rkse_list[i]]: 
                    fx_list.append(i)
            else:           
                rkse_js = round(float(row[kpse_list[i]]) - float(row[jxje_list[i]])*float(row[jxsl_list[i]]),2) #计算入库税额
    #             print(rkse_js)
                if rkse_js >= 0:           #实际入库税额与理论计算不符
                    #考虑到四舍五入之后依然有几分的差值，所以不以相等判断，以误差大于1判断
                    if abs(rkse_js - float(row[rkse_list[i]]))>1:  #4.41s
                         fx_list.append(i)
                else :        #入库税额为负时，不入库（若有入库，则风险加1）
                    if row[rkse_list[i]] != 0:
                        fx_list.append(i)
        gxfxq.append(len(fx_list))
        gxmonthq.append(len(month_list))
    df['dekjq'] = dekjq
    df['zbq']=zbq
    # df['zlq']=zlq
    df['gxfxq'] = gxfxq
    df['gxfxq'] = df['gxfxq'].astype(float)
    df['zbq']=df['zbq'].astype(float)
    # df['zlq'] = df['zlq'].astype(float)
    # df['cyrs'] = df['cyrs'].astype(float)
    # df['zczb'] = df['zczb'].astype(float)
    df['dekjq'] = df['dekjq'].astype(float)
    # 删除文字特征1
    df = df.drop(['hydl','hyzl','scjydz','bsrxm','bsrxmmp','djrq','jyfw','fdbrxmp','fddbrxm','nsrmc','zcdz','zjnsrsbh'], axis=1)
    # 删除文字特征2
    df = df.drop(['hy', 'hyml', 'xzjd'], axis=1)
    # columns = df.columns.tolist()
    tr_index = ~df['yc'].isnull()
    # 训练集的y
    y_train = df[tr_index]['yc'].reset_index(drop=True).astype(int)
    # 训练集的X
    df = df.drop(['yc'], axis=1)
    X_train = df[tr_index][df.columns].reset_index(drop=True)


    # 测试集的X
    X_test = df[~tr_index][df.columns].reset_index(drop=True)
    print("训练集的形状：",X_train.shape)
    print("测试集的形状：",X_test.shape)
    import numpy as np
    from sklearn.neural_network import MLPClassifier
    from sklearn.preprocessing import StandardScaler


    # 神经网络对数据尺度敏感，所以最好在训练前标准化，或者归一化，或者缩放到[-1,1]
    scaler = StandardScaler()      # 标准化转换,去均值和方差归一化
    scaler.fit(X_train)            # 训练标准化对象
    X_train = scaler.transform(X_train)  # 转换数据集

    # solver='lbfgs',  MLP的求解方法：L-BFGS 在小数据上表现较好，Adam 较为鲁棒，SGD在参数调整较优时会有最佳表现（分类效果与迭代次数）；SGD标识随机梯度下降。
    # alpha:L2的参数：MLP是可以支持正则化的，默认为L2，具体参数需要调整
    # hidden_layer_sizes=(5, 2) hidden层2层,第一层5个神经元，第二层2个神经元)，2层隐藏层，也就有3层神经网络

    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(5,5),random_state=41)
    # 神经网络输入为2，第一隐藏层神经元个数为5，第二隐藏层神经元个数为3，输出结果为2分类。


    clf.fit(X_train, y_train)
    X_test = scaler.transform(X_test)  # 转换数据集
    ann_predict = clf.predict_proba(X_test)
    grid_hat = ann_predict[:,1]
    test2['Probability'] = grid_hat
    prob = {
            "zjnsrsbh": test2['zjnsrsbh'],
            'Probability':test2['Probability']
        }
    verify_data = pd.DataFrame(prob)
    
    return verify_data

In [15]:
get_score.post_user_id('21')
user_verify_data = create_verify_train_data()
get_score.post_verify_data(user_verify_data)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


训练集的形状： (10000, 101)
测试集的形状： (6000, 101)
User-id:21
程序开始运行时间为：2019-12-21 23:01:45.258523
程序结束运行时间为：2019-12-21 23:02:12.837013
程序运行时间（去除打分耗时）为：27.57849
程序的准确率为：95.45475768
最终得分为：95.90928191
